### xDD Document-store API Basic Usage

The xDD document-store currently supports a simple set of operations for retrieving full documents and individual pages from the xDD corpus.

**Note**: The document-store API requires simple API key authentication. Place your API key in a file called `config.json` in this directory
prior to running the following examples.
```json
{
    "API_KEY": "<your api key>"
}
```

In [ ]:
import requests
import json
from IPython.display import FileLink, display
from ipywidgets import widgets

BASE_URL = "https://xdddev.chtc.io/documentstore-api"
XDD_API_URL="https://xdd.wisc.edu/api/articles"
with open("./config.json") as config:
    API_KEY = json.load(config)["API_KEY"]



### Find Documents

The document store API currently depends on the Main xDD API for document discoverability. Queries can be made to `https://xdd.wisc.edu/api/articles?set=criticalmaas&term=<search term>`
to perform full-text searches of documents in the critical-maas dataset.

In [ ]:
text_entry = widgets.Text(description="Search Term")
display(text_entry)

In [ ]:
results = requests.get(f"{XDD_API_URL}?dataset=criticalmaas&term={text_entry.value}&fields=title,doi&max=25").json()

doc_select = widgets.Dropdown(width=400, description="Document", options=[(d['title'],d['_gddid']) for i, d in enumerate(results['success']['data']['data'])])
display(doc_select)


#### Query Documents in the Document Store

The `/query` endpoint returns a document by xdd id or doi

In [ ]:
document_metadata = requests.get(f"{BASE_URL}/query?xdd_id={doc_select.value}").json()
doc_id = document_metadata['id']
print(json.dumps(document_metadata, indent=2))

### Retrieve the full content of an individual document

The `/documents/{document_id}/content` endpoint returns the full PDF contents of a document. Note the required `X-Api-Key` header for retrieving PDF content

In [ ]:
with open('./sample.pdf','wb') as pdf_writer:
    content = requests.get(
        f'{BASE_URL}/documents/{doc_id}/content', 
        headers={'X-Api-Key': API_KEY}
    ).content
    pdf_writer.write(content)

print(f"Document '{document_metadata['title']}' downloaded at {FileLink('./sample.pdf')}")


### Retrieve a single page from a given document

The `/documents/{document_id}/page/{page_num}` endpoint returns a single page of the specified document. 
The `content_type={pdf|webp|svg}` query parameter can be used to retrieve the page in a variety of content types.

In [ ]:
PAGE_NUM = 0
with open('./sample_page.pdf','wb') as pdf_writer:
    content = requests.get(
        f'{BASE_URL}/documents/{doc_id}/page/{PAGE_NUM}', 
        headers={'X-Api-Key': API_KEY}
    ).content
    pdf_writer.write(content)

with open('./sample_page.webp','wb') as webp_writer:
    content = requests.get(
        f'{BASE_URL}/documents/{doc_id}/page/{PAGE_NUM}?content_type=webp', 
        headers={'X-Api-Key': API_KEY}
    ).content
    webp_writer.write(content)

print(f"Document page {PAGE_NUM} downloaded at {FileLink('./sample_page.pdf')}")
print(f"Also downloaded as webp at {FileLink('./sample_page.webp')}")


### Retrieve a higlighted snippet from a given document

The `/documents/{document_id}/page/{page_num}/snippet/{x0},{y0},{x1},{y1}` endpoint returns a single page of the specified document with the given region highlighted. 
The `content_type={pdf|webp|svg}` query parameter can be used to retrieve the snippet in a variety of content types.

In [ ]:
PAGE_NUM = 0
SNIPPET = [25, 25, 725, 900]

snippet = ','.join(str(s) for s in SNIPPET)

with open('./sample_snippet.pdf','wb') as pdf_writer:
    content = requests.get(
        f'{BASE_URL}/documents/{doc_id}/page/{PAGE_NUM}/snippet/{snippet}', 
        headers={'X-Api-Key': API_KEY}
    ).content
    pdf_writer.write(content)

with open('./sample_snippet.webp','wb') as webp_writer:
    content = requests.get(
        f'{BASE_URL}/documents/{doc_id}/page/{PAGE_NUM}/snippet/{snippet}?content_type=webp', 
        headers={'X-Api-Key': API_KEY}
    ).content
    webp_writer.write(content)

print(f"Document snippet page={PAGE_NUM}; bounds={snippet} downloaded at {FileLink('./sample_snippet.pdf')}")
print(f"Also downloaded as webp at {FileLink('./sample_snippet.webp')}")
